# Gromacs simulation setup for basic protein MD

Heavily based on the famous [Lysosome tutorial](http://www.mdtutorials.com/gmx/lysozyme/).

The dashboard provides you with the molecule as `input.pdb`
First, give it a nice name, specify the main simulation length, and adjust number of equilibration steps if necessary

In [ ]:
name='protein' # give me a better name

nanoseconds = 0.05 # just for fun
nsteps = int(nanoseconds * 1000 * 500) # assumes usual 2fs integration step

# FIXME
eqsteps = 10000 # works for small proteins but can be a bit unrealistic (20ps)

## Just import what we need

In [ ]:
import gromacs as gmx
import nglview as nv
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

## Look at the input

Tune NGLView parameters if needed, inspect the input visually

In [ ]:
nv.show_file('input.pdb')

## Initial setup

### Parse the input

Let Gromacs parse the input file, specify water model and force field. Check and fix eventual errors.



In [ ]:
gmx.pdb2gmx(f='input.pdb',o=name+'.gro',water='tip3p',ff='amber99',p=name+'.top',ignh=True)

### Simulation box

Set up the box, choose it's shape and adjust size eventually.

In [ ]:
gmx.editconf(f=name+'.gro',o=name+'-box.gro',c=True,d='1.5', bt='dodecahedron')

### Solvate
Add water to the molecule.

Gromacs overwrites the topology file in this stage, therefore we rename it first.

In [ ]:
!cp {name}.top {name}-solv.top
gmx.solvate(cp=name+'-box.gro',cs='spc216.gro',o=name+'-solv.gro',p=name+'-solv.top')

### Add counterions

Add Cl- or Na+ to compensate for charge. Adjust parameters here if physiological salt concetntration is required etc.

In [ ]:
with open('ions.mdp','w') as ions:
    ions.write("""\
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme	= Verlet    ; Buffered neighbor searching 
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = cutoff    ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions
""")
    
gmx.grompp(f='ions.mdp',c=name+'-solv.gro',p=name+'-solv.top',o='ions.tpr')

In [ ]:
gmx.select(s=name+'-solv.gro',on='solv.ndx',select='SOL')

In [ ]:
!cp {name}-solv.top {name}-ions.top
gmx.genion(s='ions.tpr',n='solv.ndx',o=name+'-ions.gro',p=name+'-ions.top',pname='NA',nname='CL',neutral=True)

### Minimize

Gradient descend the energy of the system we set up so that simulation does note explode.

Visualize the minimization trajectory to check something does not go wrong.

In [ ]:
with open('minim.mdp','w') as m:
    m.write("""\
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.005          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions

nstxout                 = 50         
nstvout                 = 0        
nstfout                 = 0
nstenergy               = 50         
""")
    
gmx.grompp(f='minim.mdp',c=name+'-ions.gro',p=name+'-ions.top',o='em.tpr')

In [ ]:
gmx.mdrun(deffnm='em')

In [ ]:
gmx.select(s=name+'-ions.gro',on='protein.ndx',select='Protein')

In [ ]:
gmx.trjconv(s=name+'-ions.gro',f='em.trr',n='protein.ndx',o='em-protein.xtc')

In [ ]:
tr=md.load('em-protein.xtc',top=name+'.gro')
nv.show_mdtraj(tr)

## Equilibration

### Constant Number-Volume-Temperature

Assing some initial velocities to the atoms, restrain protein heavy atom positions, apply thermostat and run short simulation.

Observe the resulting pressure and temperature curves. They can oscillate but should not show any trend apart of an initial phase.

In [ ]:
with open('nvt.mdp','w') as nvt:
    nvt.write(f'''title                   = NVT equilibration 
define                  = -DPOSRES  ; position restrain the protein
; Run parameters
integrator              = md        ; leap-frog integrator
nsteps                  = {eqsteps}     ; 2 * 50000 = 100 ps
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 500       ; save coordinates every 1.0 ps
nstvout                 = 500       ; save velocities every 1.0 ps
nstenergy               = 500       ; save energies every 1.0 ps
nstlog                  = 500       ; update log file every 1.0 ps
; Bond parameters
continuation            = no        ; first dynamics run
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Nonbonded settings 
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is off
pcoupl                  = no        ; no pressure coupling in NVT
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Velocity generation
gen_vel                 = yes       ; assign velocities from Maxwell distribution
gen_temp                = 300       ; temperature for Maxwell distribution
gen_seed                = -1        ; generate a random seed
''')
gmx.grompp(f="nvt.mdp",c="em.gro",r="em.gro",p=name+'-ions.top',o="nvt.tpr")

In [ ]:
gmx.mdrun(deffnm='nvt',pin='on')

In [ ]:
gmx.energy(f='nvt.edr',o='press.xvg',input='Pressure')
gmx.energy(f='nvt.edr',o='temp.xvg',input='Temperature')

In [ ]:
temp = np.loadtxt('temp.xvg',comments=['#','@'])
press = np.loadtxt('press.xvg',comments=['#','@'])

plt.figure(figsize=(15,5))
plt.subplot(211)
plt.plot(press[:,0],press[:,1])
plt.title('isothermal-isochoric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")


plt.subplot(212)
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.grid()
plt.plot(temp[:,0],temp[:,1])

plt.show()

### Constant Number-Pressure-Temperature

Keep the heavy atom positions restrained, besides thermostat apply also pressure coupling.

Observer pressure, temperature, and density, which should ramp up again and remain oscillating around reasonable values.

In [ ]:
with open('npt.mdp','w') as npt:
    npt.write(f'''define                  = -DPOSRES  ; position restrain the protein
; Run parameters
integrator              = md        ; leap-frog integrator
nsteps                  = {eqsteps}     ; 2 * 50000 = 100 ps
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 500       ; save coordinates every 1.0 ps
nstvout                 = 500       ; save velocities every 1.0 ps
nstenergy               = 500       ; save energies every 1.0 ps
nstlog                  = 500       ; update log file every 1.0 ps
; Bond parameters
continuation            = yes       ; Restarting after NVT 
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Nonbonded settings 
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is on
; ljocha pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupl = C-rescale
pcoupltype              = isotropic             ; uniform scaling of box vectors
; ljocha tau_p                   = 2.0                   ; time constant, in ps
tau_p = 5.0
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
refcoord_scaling        = com
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 
''')
gmx.grompp(f="npt.mdp",c="nvt.gro",r="nvt.gro",p=name+'-ions.top',o="npt.tpr")

In [ ]:
gmx.mdrun(deffnm='npt',pin='on')

In [ ]:
gmx.energy(f='npt.edr',o='press.xvg',input='Pressure')
gmx.energy(f='npt.edr',o='dens.xvg',input='Density')
gmx.energy(f='npt.edr',o='temp.xvg',input='Temperature')

In [ ]:
temp = np.loadtxt('temp.xvg',comments=['#','@'])
press = np.loadtxt('press.xvg',comments=['#','@'])
dens = np.loadtxt('dens.xvg',comments=['#','@'])

plt.figure(figsize=(15,7))
plt.subplot(311)
plt.plot(press[:,0],press[:,1])
plt.title('isothermal-isobaric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")

plt.subplot(312)
plt.ylabel('density (kg/m3)')
plt.grid()
plt.plot(dens[:,0],dens[:,1])

plt.subplot(313)
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.grid()
plt.plot(temp[:,0],temp[:,1])

plt.show()

## Prepare production simulation

Pick the outcomes of equilibration, generate production simulation input (tpr), and proceed to the next step (performance tuning) in the Dashboard.

In [ ]:
with open('md.mdp','w') as mdp:
    mdp.write(f'''integrator              = md        ; leap-frog integrator
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 0         ; suppress bulky .trr file by specifying 
nstvout                 = 0         ; 0 for output frequency of nstxout,
nstfout                 = 0         ; nstvout, and nstfout
nstenergy               = 5000      ; save energies every 10.0 ps
nstlog                  = 5000      ; update log file every 10.0 ps
nstxout-compressed      = 5000      ; save compressed coordinates every 10.0 ps
compressed-x-grps       = Protein    
; Bond parameters
continuation            = yes       ; Restarting after NPT 
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Neighborsearching
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300 300           ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupltype              = isotropic             ; uniform scaling of box vectors
tau_p                   = 2.0                   ; time constant, in ps
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Dispersion correction
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 
nsteps = {nsteps}
''')

gmx.grompp(f="md.mdp",c="npt.gro",r="npt.gro",p=name+'-ions.top',o=name+".tpr")